In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import matplotlib.pyplot as plt
import os

# **Load data**

In [3]:
# Dữ liệu là các bài báo gồm có 3 feature là:
# Title: tiêu đề bài báo
# Sapo: tiêu đề phụ
# Body: phần nội dung báo

title, sapo, body = [], [], []
path_file = '/content/gdrive/MyDrive/Data Science/Recommendation system/News Recommendation /Data/'
for filename in os.listdir(path_file):
  with open(path_file + filename, 'r') as file_in:
    count = 0
    content = ''
    for line in file_in:
      if count == 0:
        title.append(line[:-1])
        count = count + 1
      elif count == 1:
        sapo.append(line[:-1])
        count = count + 1
      else:
        content = content + line[:-1]
    body.append(content)

In [4]:
print('Lenght of title:', len(title))
print('Lenght of sapo:', len(sapo))
print('Lenght of body:', len(body))

Lenght of title: 4419
Lenght of sapo: 4419
Lenght of body: 4419


## Create dataframe

In [5]:
df_news = pd.DataFrame({'title': title, 'sapo': sapo, 'body': body})
df_news.drop_duplicates(inplace=True)
df_news.dropna(inplace=True)
df_news.insert(0, 'id', np.arange(len(df_news)))
df_news.head()

,id,title,sapo,body
0,0,"Ly hôn chồng cũ 6 năm, vợ trẻ vẫn hoảng sợ vì ...",Suốt 6 năm qua chị liên tục nhận được những ti...,Nhiều người nói rằng phụ nữ lấy chồng chẳng kh...
1,1,Sẽ cưỡng chế tháo dỡ công trình 'xẻ thịt' rừng...,"'Thông báo để các hộ dân tự tháo dỡ, nếu không...","Tờ Zing.vn đưa tin, sáng 30/10, Chủ tịch UBND ..."
2,2,"Va chạm với xe tải, một nữ sinh nguy kịch",Đang trên đường rẽ vào đền Quả Sơn chiếc xe tả...,"Tối 30/10, Công an huyện Đô Lương (Nghệ An) ch..."
3,3,Nhà văn Kim Dung qua đời ở tuổi 94,"Nhà văn Kim Dung, tác giả của hàng loạt tác ph...","Theo thông tin được các báo Hoa ngữ đăng tải, ..."
4,4,12.000 vụ án và 2.000 bị can đang bị treo lơ l...,"Sáng nay (30/10), chất vấn tại Hội trường, ĐB ...",Nguyên nhân chủ quan chiếm tỷ lệ rất nhỏDẫn bá...


In [6]:
df_news.shape

(3616, 4)

#**Encoding data by IF-IDF**

## Importing Liberaries

In [7]:
!pip install pyvi

     |████████████████████████████████| 8.5MB 16.4MB/s 
     |████████████████████████████████| 747kB 46.8MB/s 


In [8]:
from pyvi import ViTokenizer
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Preprocessing

In [9]:
def make_lowercase(text):
  """
  Chuyển chữ hoa của text về chữ thường
  """
  return text.lower()

def remove_punctuation(text):
  """
  Xóa các dấu câu trong text
  """
  return re.sub('['+string.punctuation+']', ' ', text)

def tokenizer(text):
  """
  Tách text VD: tôi thích ăn bánh quy -> tôi thích ăn bánh_quy 
  """
  return ViTokenizer.tokenize(text)

def remove_stopwords(text):
  """
  Xóa stopwords và số
  """
  text = text.split()
  stopwords = ['bị', 'bởi', 'cả', 'các', 'cái', 'cần', 'càng', 'chỉ', 'chiếc', 'cho', 'chứ', 'chưa', 'chuyện', 
             'có', 'có_thể', 'cứ', 'của', 'cùng', 'cũng', 'đã', 'đang', 'đây', 'để', 'đến_nỗi', 'đều', 'điều', 
             'do', 'đó', 'được', 'dưới', 'gì', 'khi', 'không', 'là', 'lại', 'lên', 'lúc', 'mà', 'mỗi', 'một_cách', 
             'này', 'nên', 'nếu', 'ngay', 'nhiều', 'như', 'nhưng', 'những', 'nơi', 'nữa', 'phải', 'qua', 'ra', 
             'rằng', 'rằng', 'rất', 'rất', 'rồi', 'sau', 'sẽ', 'so', 'sự', 'tại', 'theo', 'thì', 'trên', 'trước', 
             'từ', 'từng', 'và', 'vẫn', 'vào', 'vậy', 'vì', 'việc', 'với', 'vừa']
  text = [w for w in text if not w in stopwords]
  texts = [w for w in text if not w.isdigit()]
  texts = " ".join(texts)
  return texts

df_news['body_cleaned'] = df_news['body'].apply(func=make_lowercase)
df_news['body_cleaned'] = df_news['body_cleaned'].apply(func=remove_punctuation)
df_news['body_cleaned'] = df_news['body_cleaned'].apply(func=tokenizer)
df_news['body_cleaned'] = df_news['body_cleaned'].apply(func=remove_stopwords)

In [10]:
df_news.head()

,id,title,sapo,body,body_cleaned
0,0,"Ly hôn chồng cũ 6 năm, vợ trẻ vẫn hoảng sợ vì ...",Suốt 6 năm qua chị liên tục nhận được những ti...,Nhiều người nói rằng phụ nữ lấy chồng chẳng kh...,người nói phụ_nữ lấy chồng chẳng khác_nào canh...
1,1,Sẽ cưỡng chế tháo dỡ công trình 'xẻ thịt' rừng...,"'Thông báo để các hộ dân tự tháo dỡ, nếu không...","Tờ Zing.vn đưa tin, sáng 30/10, Chủ tịch UBND ...",tờ zing vn đưa tin sáng chủ_tịch ubnd tp hà_nộ...
2,2,"Va chạm với xe tải, một nữ sinh nguy kịch",Đang trên đường rẽ vào đền Quả Sơn chiếc xe tả...,"Tối 30/10, Công an huyện Đô Lương (Nghệ An) ch...",tối công_an huyện đô lương nghệ_an biết địa_bà...
3,3,Nhà văn Kim Dung qua đời ở tuổi 94,"Nhà văn Kim Dung, tác giả của hàng loạt tác ph...","Theo thông tin được các báo Hoa ngữ đăng tải, ...",thông_tin báo hoa ngữ đăng_tải kim dung mất bệ...
4,4,12.000 vụ án và 2.000 bị can đang bị treo lơ l...,"Sáng nay (30/10), chất vấn tại Hội trường, ĐB ...",Nguyên nhân chủ quan chiếm tỷ lệ rất nhỏDẫn bá...,nguyên_nhân chủ_quan chiếm tỷ_lệ nhỏdẫn báo_cá...


## TF-IDF

In [11]:
vectorizer = TfidfVectorizer(max_df=0.8, use_idf=True, ngram_range=(1,3))
tfidf_matrix = vectorizer.fit_transform(df_news['body_cleaned'])

# **Similarity between vectors**

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
cosine_score = cosine_similarity(tfidf_matrix)
cosine_score

array([[1.        , 0.00437835, 0.00732384, ..., 0.00856488, 0.00828742,
        0.00725494],
       [0.00437835, 1.        , 0.01371953, ..., 0.00252481, 0.00582427,
        0.02857087],
       [0.00732384, 0.01371953, 1.        , ..., 0.0021417 , 0.00365852,
        0.0100892 ],
       ...,
       [0.00856488, 0.00252481, 0.0021417 , ..., 1.        , 0.00186422,
        0.00495717],
       [0.00828742, 0.00582427, 0.00365852, ..., 0.00186422, 1.        ,
        0.00578285],
       [0.00725494, 0.02857087, 0.0100892 , ..., 0.00495717, 0.00578285,
        1.        ]])

In [14]:
cosine_score.shape

(3616, 3616)

# **Recommendation**

In [16]:
def get_information(id):
  print("ID        :    ", df_news.iloc[id]['id'])
  print("Title     :    ", df_news.iloc[id]['title'])
  print("Sapo      :    ", df_news.iloc[id]['sapo'])

def recommender(id, k):
  """
  id: id của bài báo
  k: top k item cần recommender
  """
  similar_indices = cosine_score[id].argsort()[-(k+1):-1]
  get_information(id)
  print("-"*100)
  count = 1
  for i in range(k-1, -1, -1):
    print("Recommendation",count)
    get_information(similar_indices[i])
    print("Similarity:", cosine_score[id][similar_indices[i]])
    print("-"*100)
    count = count + 1

In [17]:
recommender(10, 5)

ID        :     10
Title     :     Lo lắng về sự xuống cấp của đạo đức xã hội
Sapo      :     Trong buổi chất vấn các thành viên Chính phủ sáng 30-10, nhiều đại biểu Quốc hội đã bày tỏ sự quan ngại đối với việc đạo đức xã hội, đạo đức gia đình xuống cấp hiện nay. Bộ trưởng Văn hóa, Thể thao và Du lịch trả lời chất vấn đã cho rằng, để khắc phục được tình trạng này cần thời gian, và cần sự chung tay của toàn xã hội, cũng như nhiều bộ, ngành khác.
----------------------------------------------------------------------------------------------------
Recommendation 1
ID        :     1908
Title     :     Bộ trưởng Bộ Văn hóa: Sự xuống cấp đạo đức xã hội xuất phát từ các ngành kinh tế!
Sapo      :     Trả lời chất vấn của đại biểu Quốc hội về trách nhiệm trước tình trạng đạo đức xuống cấp, Bộ trưởng Nguyễn Ngọc Thiện khẳng định: 'Sự xuống cấp đạo đức xã hội xuất phát từ các ngành kinh tế!'
Similarity: 0.5953241800323615
---------------------------------------------------------------------------